In [ ]:
#imports and preprocessing, setting up train_loader

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    cudnn.deterministic = True
    cudnn.benchmark = True
    gpu_index = 0
else:
    device = torch.device('cpu')
    gpu_index = -1



In [ ]:
# declare train_dateset and train_loader

In [ ]:
from models.classificationModel import ClassificationModel
from models.classificationTrainer import Trainer

model = ClassificationModel(out_dim=20, dataset='')



In [ ]:
optimizer = torch.optim.Adam(model.parameters(), 0.0003, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader), eta_min=0,
                                                       last_epoch=-1)

In [ ]:
with torch.cuda.device(gpu_index):
    trainer = Trainer(model=model, optimizer=optimizer, scheduler=scheduler, device=device, epochs=200, log_every_n_steps=100,
                      log_dir='test')
    trainer.train(train_loader)

In [ ]:
torch.save(model.state_dict(), 'saved_models/test.pth')